# Fast Polynomial Multiplication with DFT/FFT implementation, RSA Encryption, Image compression (20 Marks)

1. (Done) Implement 1-D DFT ,on coefficient vectors of two polynomials A(x), B(x) by multiplication of Vandermonde matrix. ( O(n 2 ) - Complexity)
2. (Done) Implement 1-D FFT on the same vectors, of A(x) and B(x). Ensure above two steps produce same results. ( O(n logn) – Complexity)
3. (Done) Pointwise multiply results of Step (2) to produce C(x) in P-V form
4. (Done) RSA encrypt (128-bit , 256-bit and 512-bit ) , with public key , the C(x) in PV form, for transmission security and decrypt with a private key and verify.
5. Implement 1-D Inverse FFT (I-FFT) on C(x), in PV form (Interpolation) to get C(x) in Coefficient form (CR) Polynomial.
6. Verify correctness of C(x) , by comparing with the coefficients generated by a Elementary “Convolution For Loop” on the Coefficients of A(x) and B(x)
7. Implement a 2-D FFT and 2-D I-FFT module using your 1-D version (This just means , applying FFT on the Rows First and Columns Next on M x N matrix of numbers !!)
8. Verify your of Step (7) correctness on a Grayscale matrix ( which has random integer values in the range 0-255; 255 → White & 0 → Black))
9. Apply your 2D-FFT on TIFF/JPG (lossless) Grayscale image and drop Fourier coefficients below some specified magnitude and save the 2D- image to a new file.
    - ( relates to % compression – permanent Lossy compression)
    - ( by sorting and retaining only coefficients greater than some(quantization) value. Rest are made 0.)
10. Apply 2D I-FFT, on the Quantized Grayscale image and render it to observe Image Quality.

In [17]:
import numpy as np
import binascii
import random

# Using scipy's fft to verify our implementation
from scipy.fft import fft as sc_fft 

## 1-D DFT

Implement 1-D DFT ,on coefficient vectors of two polynomials A(x), B(x) by multiplication of Vandermonde matrix. ( O(n 2 ) - Complexity)

In [3]:
# dft implementation
def dft(x):
    x = np.asarray(x, dtype=float)
    N = x.shape[0]
    n = np.arange(N)
    k = n.reshape((N, 1))
    M = np.exp(-2j * np.pi * k * n / N)
    return np.dot(M, x)

In [6]:
# Generate A(x), B(x)
A = np.random.random(4)
B = np.random.random(4)

print("A=", A)
print("B=", B)

A= [0.52727598 0.43217533 0.93361006 0.60872562]
B= [0.30042868 0.19569909 0.9103188  0.62298186]


In [8]:
# Run dft() on both A, B
A_dft = dft(A)
B_dft = dft(B)

print("A_dft =", A_dft)
print("B_dft =", B_dft)


A_dft = [ 2.50178699+0.00000000e+00j -0.40633408+1.76550283e-01j
  0.41998508-4.78998644e-17j -0.40633408-1.76550283e-01j]
B_dft = [ 2.02942843+0.00000000e+00j -0.60989012+4.27282764e-01j
  0.39206652-2.98822480e-17j -0.60989012-4.27282764e-01j]


In [9]:
# Verify the output by comparing to scipy's implementation
if (np.allclose(A_dft, np.fft.fft(A)) and np.allclose(B_dft, np.fft.fft(B))):
    print("\033[92mPASSED\033[0m DFT")
else:
    print("\033[91mFAILED\033[0m DFT")

PASSED DFT


## 1-D FFT

Implement 1-D FFT on the same vectors, of A(x) and B(x). Ensure above two steps produce same results. ( O(n logn) – Complexity)

In [11]:
# fft implementation

def fft(x):
    x = np.asarray(x, dtype=float)
    N = x.shape[0]
    if N % 2 > 0:
        raise ValueError("must be a power of 2")
    elif N <= 2:
        return dft(x)
    else:
        X_even = fft(x[::2])
        X_odd = fft(x[1::2])
        terms = np.exp(-2j * np.pi * np.arange(N) / N)
        return np.concatenate([X_even + terms[:int(N/2)] * X_odd,
                               X_even + terms[int(N/2):] * X_odd])


In [13]:
# Run fft() on both A, B
A_fft = fft(A)
B_fft = fft(B)

print("\033[96mINFO\033[0m A_fft =", A_fft)
print("\033[96mINFO\033[0m B_fft =", B_fft)

INFO A_fft = [ 2.50178699+0.00000000e+00j -0.40633408+1.76550283e-01j
  0.41998508-1.27473602e-16j -0.40633408-1.76550283e-01j]
INFO B_fft = [ 2.02942843+0.00000000e+00j -0.60989012+4.27282764e-01j
  0.39206652-1.00259501e-16j -0.60989012-4.27282764e-01j]


In [14]:
# Verify the output by comparing to scipy's implementation
if (np.allclose(A_fft, np.fft.fft(A)) and np.allclose(B_fft, np.fft.fft(B))):
    print("\033[92mPASSED\033[0m FFT")
else:
    print("\033[91mFAILED\033[0m FFT")

PASSED FFT


## Pointwise multiply results of Step (2) to produce C(x) in P-V form

In [16]:
C = np.multiply(A_fft, B_fft)

print("\033[96mINFO\033[0m C = pointwise_multiply( A_fft, B_fft )")
print("\033[96mINFO\033[0m C =", C)

INFO C = pointwise_multiply( A_fft, B_fft )
INFO C = [5.07719764+0.00000000e+00j 0.17238225-2.81295821e-01j
 0.16466209-9.20856269e-17j 0.17238225+2.81295821e-01j]


## RSA Implementation

Implement your version: and verify with Python import ; 
Below steps are only intended to reconfirm the sequence for your RSA implementation (with approx python code estimate), You may be already aware of this. If so, pls ignore. 

1. Generate two random Odd large numbers ( of a given bit size) - ( 4 lines) 
2. Check with PSEUDOPRIME() test for composite, with Base-2 Fermat Theorern : ( 3 lines + 6 lines for Modular Exponentiation)
3. Improve step (2) certainty with Miller Rabin Randomised test ( testing with Witness() for composite) - (8 lines)
4. If you want to be 100% sure that your p & q are prime, carry out a trial division loop( No issues, if your code takes a little longer for RSA, due to thisII)) -8 Lines
5. Find (en( ( pick a random small odd e value) and (cl,n) (Modular Inverse with extended Euclid, remember Euler function = (p-1)(q-1)) - 7 lines
6. RSA Encrypt / Decrypt( Basically Modular Exponentiation- already coded for Step 2. Only wrapper for 8-byte blocks of Data to Encrypt/Decrypt)- 10 Lines. 

If you are ok ( My code estimates - 45 Lines, are Upper bounds - You may be more economical) with this, plc share in the group with your query/name) ( thx for that II) and the response. Many teams may have the same query. 


In [21]:
"""
Implementation of the RSA algorithm.
It randomly selects two prime numbers from a txt file of prime numbers and 
uses them to produce the public and private keys. Using the keys, it can 
either encrypt or decrypt messages.
"""

def gcd(a, b):
    """
    Performs the Euclidean algorithm and returns the gcd of a and b
    """
    if (b == 0):
        return a
    else:
        return gcd(b, a % b)

def xgcd(a, b):
    """
    Performs the extended Euclidean algorithm
    Returns the gcd, coefficient of a, and coefficient of b
    """
    x, old_x = 0, 1
    y, old_y = 1, 0

    while (b != 0):
        quotient = a // b
        a, b = b, a - quotient * b
        old_x, x = x, old_x - quotient * x
        old_y, y = y, old_y - quotient * y

    return a, old_x, old_y

def chooseE(totient):
    """
    Chooses a random number, 1 < e < totient, and checks whether or not it is 
    coprime with the totient, that is, gcd(e, totient) = 1
    """
    while (True):
        e = random.randrange(2, totient)

        if (gcd(e, totient) == 1):
            return e

def chooseKeys():
    """
    Selects two random prime numbers from a list of prime numbers which has 
    values that go up to 100k. It creates a text file and stores the two 
    numbers there where they can be used later. Using the prime numbers, 
    it also computes and stores the public and private keys in two separate 
    files.
    """

    # choose two random numbers within the range of lines where 
    # the prime numbers are not too small and not too big
    rand1 = random.randint(100, 300)
    rand2 = random.randint(100, 300)

    # store the txt file of prime numbers in a python list
    fo = open('primes-to-100k.txt', 'r')
    lines = fo.read().splitlines()
    fo.close()

    # store our prime numbers in these variables
    prime1 = int(lines[rand1])
    prime2 = int(lines[rand2])

    # compute n, totient, e
    n = prime1 * prime2
    totient = (prime1 - 1) * (prime2 - 1)
    e = chooseE(totient)

    # compute d, 1 < d < totient such that ed = 1 (mod totient)
    # e and d are inverses (mod totient)
    gcd, x, y = xgcd(e, totient)

    # make sure d is positive
    if (x < 0):
        d = x + totient
    else:
        d = x

    # write the public keys n and e to a file
    f_public = open('public_keys.txt', 'w')
    f_public.write(str(n) + '\n')
    f_public.write(str(e) + '\n')
    f_public.close()

    f_private = open('private_keys.txt', 'w')
    f_private.write(str(n) + '\n')
    f_private.write(str(d) + '\n')
    f_private.close()
    
    return n, e, d

def encrypt(message, file_name = 'public_keys.txt', block_size = 2):
    """
    Encrypts a message (string) by raising each character's ASCII value to the 
    power of e and taking the modulus of n. Returns a string of numbers.
    file_name refers to file where the public key is located. If a file is not 
    provided, it assumes that we are encrypting the message using our own 
    public keys. Otherwise, it can use someone else's public key, which is 
    stored in a different file.
    block_size refers to how many characters make up one group of numbers in 
    each index of encrypted_blocks.
    """

    try:
        fo = open(file_name, 'r')

    # check for the possibility that the user tries to encrypt something
    # using a public key that is not found
    except FileNotFoundError:
        print('That file is not found.')
    else:
        n = int(fo.readline())
        e = int(fo.readline())
        fo.close()

        encrypted_blocks = []
        ciphertext = -1

        if (len(message) > 0):
            # initialize ciphertext to the ASCII of the first character of message
            ciphertext = ord(message[0])

        for i in range(1, len(message)):
            # add ciphertext to the list if the max block size is reached
            # reset ciphertext so we can continue adding ASCII codes
            if (i % block_size == 0):
                encrypted_blocks.append(ciphertext)
                ciphertext = 0

            # multiply by 1000 to shift the digits over to the left by 3 places
            # because ASCII codes are a max of 3 digits in decimal
            ciphertext = ciphertext * 1000 + ord(message[i])

        # add the last block to the list
        encrypted_blocks.append(ciphertext)

        # encrypt all of the numbers by taking it to the power of e
        # and modding it by n
        for i in range(len(encrypted_blocks)):
            encrypted_blocks[i] = str((encrypted_blocks[i]**e) % n)

        # create a string from the numbers
        encrypted_message = " ".join(encrypted_blocks)

        return encrypted_message

def decrypt(blocks, block_size = 2):
    """
    Decrypts a string of numbers by raising each number to the power of d and 
    taking the modulus of n. Returns the message as a string.
    block_size refers to how many characters make up one group of numbers in
    each index of blocks.
    """

    fo = open('private_keys.txt', 'r')
    n = int(fo.readline())
    d = int(fo.readline())
    fo.close()

    # turns the string into a list of ints
    list_blocks = blocks.split(' ')
    int_blocks = []

    for s in list_blocks:
        int_blocks.append(int(s))

    message = ""

    # converts each int in the list to block_size number of characters
    # by default, each int represents two characters
    for i in range(len(int_blocks)):
        # decrypt all of the numbers by taking it to the power of d
        # and modding it by n
        int_blocks[i] = (int_blocks[i]**d) % n
        
        tmp = ""
        # take apart each block into its ASCII codes for each character
        # and store it in the message string
        for c in range(block_size):
            tmp = chr(int_blocks[i] % 1000) + tmp
            int_blocks[i] //= 1000
        message += tmp

    return message

In [23]:
n, e, d = chooseKeys()

print('public_keys')
print('\t', str(n))
print('\t', str(e))

print('private_keys')
print('\t', str(n))
print('\t', str(d))

public_keys
	 1005779
	 841501
private_keys
	 1005779
	 688501


In [26]:
message = "Secret Message"

In [27]:
print('Encrypting...')
encrypted_message = encrypt(message)
print(encrypted_message)

Encrypting...
43632 152193 190053 500601 869552 99720 271865


In [29]:
print('Decryption...')
decrypted_message = decrypt(encrypted_message)
print(decrypted_message)

Decryption...
Secret Message
